# Regresión logística

Hasta ahora, hemos construído modelos supervisados de **regresión**, es decir, modelos en los que la salida de la función que aprendemos es numérica. Esta semana haremos nuestros primeros modelos de **clasificación**, en los que la salida es categórica, utilizando regresión logística.

Véamos un ejemplo. Tenemos un dataset con dos columnas, `horas_estudio` y `resultado_examen`. `horas_estudio` es una variable numérica que representa el número de horas que un estudiante le ha dedicado a prepararse para un examen, mientras que `resultado_examen` es una variable binaria que puede tomar uno de los varlores `APROBADO` o `REPROBADO`. 

| `horas_estudio` | `resultado_examen` |
| --------------- | ------------------ |
| 0 | `REPROBADO` |
| 7 | `APROBADO` |
| 3 | `REPROBADO` |
| 4 | `APROBADO` |
| ... | ... |
| 2 | `REPROBADO` |

Nuestro objetivo es **crear un modelo para determinar si un estudiante aprobará o no el examen**, utilizando como entrada el número de horas que ha dedicado a prepararse. Específicamente, aprenderemos una función $f(\texttt{horas}\_\texttt{estudio}) \mapsto \left[0, 1\right]$ que devuelve la probabilidad de que el resultado del examen sea `APROBADO`. 

In [ ]:
# Ejecutemos esta celda para cargar el dataset y las bibliotecas básicas
import matplotlib.pyplot as plt
import numpy as np
import math

horas_estudio = [5, 4, 1, 1, 1, 2, 3, 3, 2, 0, 3, 3, 2, 3, 2, 3, 1, 5, 4, 4, 3, 4, 2, 4, 3, 3, 6, 4, 4, 5, 3, 4, 3, 2, 6, 2, 4, 4, 1, 5, 3, 3, 5, 7, 4, 3, 5, 2, 3, 5, 3, 4, 3, 3, 1, 3, 6, 0, 4, 3, 7, 2, 5, 3, 3, 4, 3, 1, 5, 4, 1, 4, 3, 1, 2, 4, 2, 2, 2, 2, 4, 3, 3, 1, 4, 2, 1, 0, 4, 4, 3, 3, 4, 6, 5, 3, 2, 2, 6, 6]
resultado_examen = ['APROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'APROBADO', 'REPROBADO', 'REPROBADO', 'APROBADO', 'APROBADO']

### Notas importantes

- El nombre _regresión_ logística suele confundir. Estrictamente *sí* es un modelo de regresión que modela una variable numérica (la probabilidad), pero lo utilizamos para construir un clasificador
- La regresión logística es un **modelo probabilístico** (la función que aprendemos retorna una probabilidad). Otros modelos, p.ej. los árboles de decisión, producen directamente las etiquetas
- La regresión logística modela una salida binaria, p.ej. `APROBADO`/`REPROBADO`, `TRUE`/`FALSE`. Para más de dos categorías, el modelo general es la **regresión Softmax** que estudiaremos más adelante
- Un detalle importante en un modelo probabilístico es el "threshold" o **valor que marca la frontera entre una etiqueta de clasificación y la otra**. Esto lo estudiaremos con más detalle en el notebook sobre curvas ROC. Por ahora, fijaremos este valor en 0.5 ya que es el número en el que la probabilidad de una clase domina a la otra.  

## Ingredientes para regresión logística

Usaremos como base la función lineal de regresión $f(x) = \alpha x + \beta$, con dos ingredientes adicionales:

1. Una función de enlace o activación que nos garantiza que la salida estará entre 0 y 1
2. Una función de error para minimizar, adecuada al tipo de datos binario de la salida.

### La función logística

Existe una familia entera de funciones llamadas sigmoides. Se les llama de esta manera porque al graficarlas producen una **línea en forma de "S"**. La función logística es una de ellas. De hecho, en mucha de la literatura de Machine Learning se le conoce simplemente como "función sigmoide". Se define de la siguiente manera: $$\mathrm{sig} \left(x \right) = \frac{1}{1 + e^{-x}}$$

Su derivada con es $$\mathrm{sig}' \left(x \right) = \mathrm{sig} \left(x \right) \left(1 - \mathrm{sig} \left( x \right) \right)$$


In [ ]:
# Definamos la función logística (sigmoide) y su derivada
def sig(x):
    return 1.0 / (1.0 + np.exp(-x))

def d_sig(x):
    return sig(x) * (1.0 - sig(x))

In [ ]:
# Grafiquemos la función logística. 
# Notemos cómo el valor del eje 'y' está entre 0 y 1. Notemos también que sig(0)=0.5
x_val = np.linspace(-10, 10)
plt.plot(x_val, sig(x_val))

### La función de entropía cruzada

El segundo ingrediente para hacer regresión logística es tener una función de error que tome el lugar de MSE y sea **más adecuada para una respuesta binaria**. Para esto, denotemos la variable $y$ como un valor original obtenido del dataset y supongamos que dichos valores han sido codificados adecuadamente (siguiendo nuestro ejemplo, `1="APROBADO"` y `0="REPROBADO"`). 

In [ ]:
# Codifiquemos la variable 'resultado_examen' de modo que APROBADO=1.0 y REPROBADO=0.0
resultado_examen_enc = np.array([1.0 if y == 'APROBADO' else 0.0 for y in resultado_examen])

De manera similar, denotemos la variable $\hat{y}$ como una predicción realizada por nuestro modelo. A diferencia de $y$, que puede tomar solo los valores 0 o 1, $\hat{y}$ puede tomar cualquier valor en el intervalo $[0,1]$. Queremos **cuantificar el error de predecir $\hat{y}$ cuando el valor real es $y$**, así $$\mathrm{bce}\left(y, \hat{y} \right)=\begin{cases}-\textrm{log}(\hat{y}) &\textrm{si } y=1 \\ -\textrm{log}(1 - \hat{y}) &\textrm{si } y=0\end{cases}$$

**Nota**: aquí $\textrm{log}$ se refiere a la función de logaritmo natural. Para referirnos a la función logística, usaremos el término $\textrm{sig}$.

Tratemos de entender lo que sucede aquí. Cuando el valor real $y=1$, el error de nuestra predicción $\hat{y}$ tenderá a cero a medida que $\hat{y} \rightarrow 1$, al mismo tiempo que tenderá a infinito a medida que $\hat{y} \rightarrow 0$. Lo mismo sucede vice-versa cuando $y=0$.

In [ ]:
# Probemos la función de error de entropía cruzada con distintos valores
def bce_simple(y_real, y_pred):
    if math.isclose(y_real, 1.0):
        return -math.log(y_pred)
    
    return -math.log(1.0 - y_pred)

for rv in [1.0, 0.0]:
    print(' ')
    for tv in [0.99, 0.8, 0.5, 0.2, 0.01]:
        print('Cuando y_real =', rv, '| y_pred =', tv, '\t--> error =', bce_simple(rv, tv))

Esta función de error se puede representar también de la forma $$\mathrm{bce}(y, \hat{y}) = -y \mathrm{log}(\hat{y}) - (1-y) \mathrm{log}(1 - \hat{y})$$
siendo su derivada con respecto a $\hat{y}$ $$\frac{\partial \mathrm{bce}(y, \hat{y})}{\partial \mathrm{\hat{y}}} = \frac{\hat{y} - y}{\hat{y}(1-\hat{y})}.$$

Para efectos de un dataset compuesto por $N$ observaciones, la función de error sería $$\mathrm{bce}(\mathbf{y}, \mathbf{\hat{y}}) = \sum_{i=1}^{N} -y_i \mathrm{log}(\hat{y_i}) - (1-y_i) \mathrm{log}(1 - \hat{y_i})$$

In [ ]:
# Definamos la función de error de entropía cruzada para dataset enteros representados como arreglos de numpy
def bce(y_real, y_pred):
    return np.sum(-y_real*np.log(y_pred) - (1.0 - y_real)*np.log(1.0 - y_pred))

def d_bce(y_real, y_pred):
    return (y_pred - y_real) / (y_pred * (1.0-y_pred))

### Mezclando los ingredientes

Ya hemos definido los dos ingredientes adicionales para hacer regresión logística. Volvamos al planteamiento inicial sobre la ecuación de regresión $f(x)= \alpha x + \beta$. En regresión logística, extendemos esta ecuación con la función logística, teniendo así el modelo $$g(x) = \mathrm{sig} \left( \alpha x + \beta \right).$$
Nuestro objetivo de optimización es **encontrar los valores de $\alpha$ y $\beta$** que minimizan el valor de la función objetivo $\mathbf{bce}$ sobre un dataset dado.

## Estimación de parámetros 

Volvamos a nuestro ejemplo del exámen y las horas de estudio.

In [ ]:
# Definamos nuestro diccionario global de parámetros.
dic_param = {
    'alpha': 0.0,
    'beta': 0.0,
}

# Definamos nuestro diccionario global de constantes
dic_const = {
    'threshold': 0.5, # frontera límite de clasificación--si valor es mayor, clasifica como APROBADO.
    'lr': 0.01, # tasa de aprendizaje para gradiente descendiente
    'num_iter': 500 # número de iteraciones
}

# Retorna la probabilidad de APROBADO
def prob_aprobado(horas_estudio_x):    
    return sig(horas_estudio_x * dic_param['alpha'] + dic_param['beta'])

# Retorna la etiqueta APROBADO/REPROBADO
def label_aprobado(horas_estudio):
    return 'APROBADO' if prob_aprobado(horas_estudio) > dic_const['threshold'] else 'REPROBADO'

Para encontrar los valores de $\alpha$ y $\beta$, utilizaremos el algoritmo de gradiente descendiente. La manera estándar de hacerlo es encontrando las derivadas de la función de costo contra $\alpha$ y $\beta$ utilizando la regla de la cadena de cálculo: 
$$\frac{\partial \mathrm{bce}(y, \hat{y})}{\partial \alpha} = \frac{\partial \mathrm{bce}(y, \hat{y})}{\partial \hat{y}} \times \frac{\partial \hat{y}}{\partial f} \times \frac{\partial f}{\partial \alpha}$$
y del mismo modo con $\beta$. Lo interesante es que **nosotros no tenemos que encontrar esta derivada simbólicamente**, sino simplemente multiplicar los valores numéricos resultantes de evaluar por separado las derivadas de las funciones de regresión, error y activación, teniendo estas últimas dos ya implementadas como ``d_sig`` y ``d_bce``. Como veremos más adelante, este "truco" es la base del algoritmo *backpropagation* que se utiliza para encontrar los parámetros de las redes neuronales. Aunque nos falta implementar las derivadas de $f(x)$ para $\alpha$ y $\beta$, estas son $x$ y $1$, respectivamente. 

In [ ]:
# Función para calcular los gradientes de alpha y beta, los devuelve como una tupla de la forma (alpha,beta)
# Recibe el dataset (x, y_real) y los valores actuales de alpha y beta
def grads(x, y_real, cur_alpha, cur_beta):
    
    # Calculamos el valor la función de regresión
    f_val = x * cur_alpha + cur_beta
    
    # Generamos nuestra predicción
    y_pred = sig(f_val)
    
    # Obtenemos el gradiente del error de dicha predicción
    d_err = d_bce(y_real, y_pred)
    
    # Obtenemos el gradiente de la logística 
    d_f_val = d_sig(f_val)
    
    # Y los gradientes con respecto a alpha y beta usando la regla de la cadena
    # Los sumamos para obtener un gradiente escalar que podemos utilizar en nuestra actualización de parámetros
    d_alpha = np.sum(d_err * d_f_val * x)
    d_beta = np.sum(d_err * d_f_val)
    
    return (d_alpha, d_beta)

¡Ya tenemos todo listo para armar nuestro algoritmo de gradiente descendiente!

In [ ]:
def gd(x, y):
    
    # Inicializamos alpha y beta utilizando una distribución normal estándar
    [alpha, beta] = np.random.randn(2)
    lr = dic_const['lr']
    
    # Lista de error por iteración (para graficar)
    errs = []
    
    for i in range(dic_const['num_iter']):
        
        # Calculamos las predicciones y el error para todo el dataset
        pred = sig(alpha*x + beta)
        err = bce(y, pred)
        
        # Calculamos los gradientes
        (d_alpha, d_beta) = grads(x, y, alpha, beta)
        
        # Actualizamos los parámetros usando los gradientes y la tasa de aprendizaje
        alpha = alpha - lr*d_alpha
        beta = beta - lr*d_beta
        
        # Guardamos el error de esta iteración
        errs.append(err)
        
    
    # Actualizamos los valores de los parámetros para salida
    dic_param['alpha'] = alpha
    dic_param['beta'] = beta
    
    return errs

In [ ]:
# Ejecutamos nuestra optimización con descenso de gradientes y graficamos la curva de error entre iteraciones
errs = gd(np.array(horas_estudio), resultado_examen_enc)
plt.plot(errs)
plt.show()

# Probamos nuestro modelo contra distintas horas de estudio y graficamos la probabilidad
hora_test = np.linspace(0, 10)
hora_prob = prob_aprobado(hora_test)
plt.plot(hora_test, hora_prob)
plt.show()

## Regresión logística multivariable

Ya que tenemos nuestro primer modelo, vamos a hacer una extensión importante: multiples variables de entrada. Supongamos, que además de ``horas_estudio``, tenemos otras variables adicionales en nuestro dataset: ``horas_sueno`` y ``num_repeticiones`` que representan respectivamente el número de horas de sueño antes de tomar el examen, y el número de veces que el estudiante ha tomado el examen anteriormente (``0`` si es la primera vez que lo toma). De modo que ahora la función que debemos aprender es $f\left(\texttt{horas}\_\texttt{estudio}, \texttt{horas}\_\texttt{sueno}, \texttt{num}\_\texttt{repeticiones} \right) \mapsto \left[0, 1 \right]$.

Esto cambia el problema de varias maneras:

1. $\mathbf{x}$ en nuestro dataset pasa de ser un vector de $N$ valores, a una matriz $X$ de $N \times 3$
2. $\alpha$ pasa de ser un valor escalar y se convierte en un vector $\boldsymbol{\alpha}$ de $3$ valores.
3. Nuestro modelo es ahora $g(\mathbf{x}) = \mathrm{sig} \left(\alpha_1 x_1 + \alpha_2 x_2 + \alpha_3 x_3 + \beta \right) = \mathrm{sig}\left(\boldsymbol{\alpha}^{\mathrm{T}}\mathbf{x} + \beta \right)$

Veamos primero como quedan nuestros datos

In [ ]:
# Dataset
horas_sueno = [4, 4, 5, 5, 4, 6, 7, 7, 5, 8, 5, 7, 7, 7, 7, 7, 4, 8, 4, 8, 6, 8, 5, 4, 5, 5, 7, 4, 6, 8, 6, 4, 7, 4, 5, 7, 7, 4, 6, 6, 5, 6, 7, 6, 4, 8, 7, 4, 4, 6, 5, 8, 6, 8, 4, 5, 4, 6, 8, 5, 6, 6, 5, 6, 4, 8, 4, 6, 5, 7, 5, 6, 7, 7, 8, 4, 4, 6, 8, 6, 4, 6, 5, 5, 5, 4, 4, 7, 8, 5, 4, 8, 5, 6, 5, 6, 7, 7, 4, 5]
repeticiones = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# Combinamos los tres arreglos 'horas_estudio', 'horas_sueno' y 'repeticiones' en una matriz de Nx3
datos_mul = np.array((horas_estudio, horas_sueno, repeticiones)).transpose()

# Definimos un diccionario de parámetros nuevo para el caso multivariable
dic_param_mul = {
    'alpha': np.array([0.0, 0.0, 0.0]), # ahora es un vector
    'beta': 0.0 # permanece igual
}

# Definamos nuestro diccionario global de constantes
dic_const_mul = {
    'threshold': 0.5, # frontera límite de clasificación--si valor es mayor, clasifica como APROBADO.
    'lr': 0.001, # tasa de aprendizaje para gradiente descendiente -- MAS PEQUEÑA EN ESTE CASO
    'num_iter': 1000 # número de iteraciones -- MÁS ALTA EN ESTE CASO
}


El cambio más importante viene a la hora de aplicar la función de regresión. Para poder multiplicar el vector $\boldsymbol{\alpha}$ contra el vector de valores $\mathbf{x}$ y producir un valor escalar, utilizaremos la función ``numpy.dot``. Este cambio lo debemos hacer en nuestra función de evaluación de gradientes y en la de optimización gradiente descendiente.

In [ ]:
# Función para calcular los gradientes de alpha y beta, los devuelve como una tupla de la forma (alpha,beta)
# Recibe el dataset (x, y_real) y los valores actuales de alpha y beta
def grads_mul(x, y_real, cur_alpha, cur_beta):
    
    # Calculamos el valor la función de regresión
    # OBSERVEMOS ESTE CAMBIO contra la implementación anterior
    ### ANTES: f_val = x * cur_alpha + cur_beta
    ### AHORA:
    f_val = np.dot(x, cur_alpha) + cur_beta
    
    # Generamos nuestra predicción
    y_pred = sig(f_val)
    
    # Obtenemos el gradiente del error de dicha predicción
    d_err = d_bce(y_real, y_pred)
    
    # Obtenemos el gradiente de la logística 
    d_f_val = d_sig(f_val)
    
    # Y los gradientes con respecto a alpha y beta usando la regla de la cadena
    # Los sumamos para obtener un gradiente escalar que podemos utilizar en nuestra actualización de parámetros
    ### ANTES: d_alpha = np.sum(d_err * d_f_val * x)
    ### AHORA:
    d_alpha = np.dot(d_err * d_f_val, x)
    d_beta = np.sum(d_err * d_f_val)
    
    return (d_alpha, d_beta)

# Ahora cambiemos la función de obtención de gradientes
def gd_mul(x, y):
    
    # Inicializamos alpha y beta utilizando una distribución normal estándar
    # CAMBIO: los inicializamos ahora por separado
    alpha = np.random.randn(3)
    [beta] = np.random.randn(1)
    lr = dic_const_mul['lr']
    
    # Lista de error por iteración (para graficar)
    errs = []
    
    for i in range(dic_const_mul['num_iter']):
        
        # Calculamos las predicciones y el error para todo el dataset
        # CAMBIO: función de regresión usando np.dot
        ### ANTES: pred = sig(alpha*x + beta)
        pred = sig(np.dot(x, alpha) + beta)
        err = bce(y, pred)
        
        # Calculamos los gradientes
        ### CAMBIO: utilizamos 'grads_mul' en vez de 'grads'
        (d_alpha, d_beta) = grads_mul(x, y, alpha, beta)
        
        # Actualizamos los parámetros usando los gradientes y la tasa de aprendizaje
        alpha = alpha - lr*d_alpha
        beta = beta - lr*d_beta
        
        # Guardamos el error de esta iteración
        errs.append(err)

    # Actualizamos los valores de los parámetros para salida
    dic_param_mul['alpha'] = alpha
    dic_param_mul['beta'] = beta
    
    return errs

In [ ]:
# Ejecutamos nuestro algoritmo de optimización y graficamos el resultado
errs = gd_mul(datos_mul, resultado_examen_enc)
plt.plot(errs)